<a href="https://colab.research.google.com/github/JuliethLopez/chatbot_simpsons/blob/master/Generador%20de%20texto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatbot de los Simpsons

Con éste código se pretende hacer un chatbot a partir de la generación de texto con un red neuronal recurrente, particularmente una red GRU. El modelo debe aprender a generar caracteres para formar palabras con sentido y con ello una oración que corresponda a una palabra o una frase.

Para comenzar cargaremos las librerias y la base de datos:

In [51]:
#librerias
import tensorflow as tf #para el uso de keras y el manejo de tensores.
import numpy as np #para manejo de bases de datos
import os #para leer y escribir archivos
import time

print("Versión de Tensorflow: ", tf.__version__)

Versión de Tensorflow:  2.2.0


La base de datos fue tomada del conjunto de bases de datos __The Simpsons by the Data__ en [data.world](https://data.world/data-society/the-simpsons-by-the-data) (*para usar esta página es necesario registrarse*). Éste contiene distintas bases de datos que con los personajes, las locaciones, los detalles de los episodios, y los dialogos de aproximadamente 600 episodios de los simpsons. Utilizamos la base de datos *simpsons_script_lines.csv* que inicialmente tenia 158271 lineas de conversación, por ejemplo "Lisa Simpson: Challenge accepted!" y la filtramos por la variable raw_text=TRUE usando SQL desde la misma página [data.world](https://data.world/data-society/the-simpsons-by-the-data), obteniendo 132113 lineas.

    SELECT raw_text
    FROM simpsons_script_lines
    WHERE speaking_line = 'Guess what. I also play Frankenstein!' OR speaking_line = TRUE
    
Descargamos el archivo en formato XLSX y lo transformamos a formato TXT, no descargamos los datos en formato CSV porque aparecián caracteres no imprimibles (¦ â € Ã ©).

In [0]:
#datos
text = open('./Datos/simpsons_script_lines.txt', 'rb').read().decode(encoding='utf-8')

In [5]:
#primeros 200 caracteres
print(text[:250])

Miss Hoover: No, actually, it was a little of both. Sometimes when a disease is in all the magazines and all the news shows, it's only natural that you think you have it.
Lisa Simpson: (NEAR TEARS) Where's Mr. Bergstrom?
Miss Hoover: I don't know. 


Contamos el numero de caracteres porque el modelo se entrenara desde los caracteres, no desde las palabras como veremos más adelante.

In [6]:
print ('Length of text: {} characters'.format(len(text)))

Length of text: 9913945 characters


## 1. Vocabulario o alfabeto

Si bien los datos están en inglés, hay algunas palabras en otros idiomas debido a la complejitud de los personajes de la serie, por eso hemos decidido dejar los carácteres que pertenecen a esos otro idiomas.

In [7]:
vocab = sorted(set(text)) #set extrae caracteres distintos, sorted los ordena
print ('{} carácteres únicos'.format(len(vocab)))

138 carácteres únicos


Dentro de los caracteres tenemos \n que se utiliza como indicador para el salto de linea y \r que se utiliza para regresar del final de la linea al inicio de la linea. Pero por otro lado tenemos un caracter no deseable \xad que vamos a eliminar.

In [0]:
text = text.replace('\N{SOFT HYPHEN}', '') #elimina \xad y eliminaria mas caracteres suaves si hubieran.

In [9]:
vocab = sorted(set(text)) #set extrae caracteres distintos, sorted los ordena
print ('{} carácteres únicos'.format(len(vocab)))

137 carácteres únicos


## 2. Diccionarios de ida y vuelta
Ahora que tenemos el vocabulario de caracteres podemos crear los diccionarios de ida y vuelta que lo que harán es darle un id a cada caracter ya que para la maquina es más facil trabajar con números, de modo que cuando una frase entra, se traduce a numeros cada uno de los caracteres y se predicen los id, que luego seran transformados a carácteres para poder leerlos.

In [0]:
charxid = {u:i for i, u in enumerate(vocab)} #le damos un indice a cada caractér
text_as_int = np.array([charxid[i] for i in text]) #caracteres a números

In [11]:
#id de los primeros 10 caracteres
for char,_ in zip(charxid, range(10)):
    print('{:4s}: {:3d},'.format(repr(char), charxid[char])) #s: str, d: int

'\n':   0,
'\r':   1,
' ' :   2,
'!' :   3,
'#' :   4,
'$' :   5,
'%' :   6,
'&' :   7,
"'" :   8,
'(' :   9,


In [12]:
#primeros 15 carácteres del texto a enteros
print ('{} ---- carácteres a enteros ---- > {}'.format(repr(text[:15]), text_as_int[:15]))

'Miss Hoover: No' ---- carácteres a enteros ---- > [42 69 79 79  2 37 75 75 82 65 78 27  2 43 75]


In [13]:
idxchar = np.array(vocab) #números a caracteres
print("\nidxchar:", idxchar)


idxchar: ['\n' '\r' ' ' '!' '#' '$' '%' '&' "'" '(' ')' '*' '+' ',' '-' '.' '/' '0'
 '1' '2' '3' '4' '5' '6' '7' '8' '9' ':' ';' '?' 'A' 'B' 'C' 'D' 'E' 'F'
 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W' 'X'
 'Y' 'Z' '[' '\\' ']' '_' '`' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k'
 'l' 'm' 'n' 'o' 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z' '¡' '¿' 'À'
 'Ä' 'È' 'É' 'Ñ' 'Ö' 'Ù' 'Ü' 'à' 'á' 'â' 'ã' 'ä' 'å' 'æ' 'ç' 'è' 'é' 'ê'
 'ë' 'ì' 'í' 'ï' 'ñ' 'ò' 'ó' 'ô' 'õ' 'ö' 'ø' 'ù' 'ú' 'û' 'ü' 'ā' 'Ĉ' 'ē'
 'ě' 'Ĝ' 'ī' 'Ĭ' 'ł' 'ń' 'ŭ' 'ż' 'ǎ' 'ǐ' 'ǒ']


## 3. Datos de entrenamiento y etiquetas

Los datos que tenemos aún no están listos para usarse en el modelo, pues no están etiquetados. Para entender como etiquetar los datos usemos esta frase como ejemplo "Miss Hoover: No, actually, it was a little of both.". El modelo que vamos a utilizar necesita una frase de entrada, tomemos "Miss Hoover: No, actually, it was a littl" y necesita una etiqueta, en este caso es "iss Hoover: No, actually, it was a little", así:


|                     Entrada                 |  Etiqueta   |
|---------------------------------------------|:---:|
| "Miss Hoover: No, actually, it was a littl" | "iss Hoover: No, actually, it was a little" |
| "iss Hoover: No, actually, it was a little" | "ss Hoover: No, actually, it was a little " |
| "ss Hoover: No, actually, it was a little " | "s Hoover: No, actually, it was a little o" |
| "s Hoover: No, actually, it was a little o" | " Hoover: No, actually, it was a little of" |

Entonces, empezaremos por definir el tamañao de las entradas y las etiquetas.

In [14]:
seq_length = 100 #máxima longitud de la frase
#examples_per_epoch = len(text)//(seq_length+1)
examples_per_epoch = len(text)-seq_length #numero de datos por epoca
examples_per_epoch

9913834

Los datos necesitan ser transformados a tensores para que puedan ser utilizados en el modelo, los transformaremos con *tf.data.Dataset.from_tensor_slices*.

In [15]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int) #se tranforma de numpy a tf
print("Tensor:", char_dataset)

#ejemplo
for i in char_dataset.take(11):
  print(idxchar[i.numpy()])

Tensor: <TensorSliceDataset shapes: (), types: tf.int64>
M
i
s
s
 
H
o
o
v
e
r


Ahora podemos construir las secuencias de entrenamiento, tengamos en cuenta que las estas se componen por seq_length caracteres mas un caracter que es el que se usara para crear la etiqueta.

In [16]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
print("sequences:", sequences)

for item in sequences.take(2):
  print(repr(idxchar[item.numpy()])) #rep vuelve imprimible lo que es imprimible

sequences: <BatchDataset shapes: (101,), types: tf.int64>
array(['M', 'i', 's', 's', ' ', 'H', 'o', 'o', 'v', 'e', 'r', ':', ' ',
       'N', 'o', ',', ' ', 'a', 'c', 't', 'u', 'a', 'l', 'l', 'y', ',',
       ' ', 'i', 't', ' ', 'w', 'a', 's', ' ', 'a', ' ', 'l', 'i', 't',
       't', 'l', 'e', ' ', 'o', 'f', ' ', 'b', 'o', 't', 'h', '.', ' ',
       'S', 'o', 'm', 'e', 't', 'i', 'm', 'e', 's', ' ', 'w', 'h', 'e',
       'n', ' ', 'a', ' ', 'd', 'i', 's', 'e', 'a', 's', 'e', ' ', 'i',
       's', ' ', 'i', 'n', ' ', 'a', 'l', 'l', ' ', 't', 'h', 'e', ' ',
       'm', 'a', 'g', 'a', 'z', 'i', 'n', 'e', 's', ' '], dtype='<U1')
array(['a', 'n', 'd', ' ', 'a', 'l', 'l', ' ', 't', 'h', 'e', ' ', 'n',
       'e', 'w', 's', ' ', 's', 'h', 'o', 'w', 's', ',', ' ', 'i', 't',
       "'", 's', ' ', 'o', 'n', 'l', 'y', ' ', 'n', 'a', 't', 'u', 'r',
       'a', 'l', ' ', 't', 'h', 'a', 't', ' ', 'y', 'o', 'u', ' ', 't',
       'h', 'i', 'n', 'k', ' ', 'y', 'o', 'u', ' ', 'h', 'a', 'v', 'e',
       

Juntemos los caracteres para que sean mas facil de visualizar.

In [17]:
for item in sequences.take(2):
  print(repr(''.join(idxchar[item.numpy()])))

'Miss Hoover: No, actually, it was a little of both. Sometimes when a disease is in all the magazines '
"and all the news shows, it's only natural that you think you have it.\r\nLisa Simpson: (NEAR TEARS) Whe"


Aquí dividiremos las secuencias en entradas y etiquetas con la misma longitud. El método *.map* nos permite aplicar una función simple a cada lote, similar a la función apply de R, se busca duplicar cada secuencia y cambiarla para formar el texto de entrada y de salida.

In [18]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)
dataset

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>

In [19]:
for item in dataset.take(1):
  print(item)

(<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([42, 69, 79, 79,  2, 37, 75, 75, 82, 65, 78, 27,  2, 43, 75, 13,  2,
       61, 63, 80, 81, 61, 72, 72, 85, 13,  2, 69, 80,  2, 83, 61, 79,  2,
       61,  2, 72, 69, 80, 80, 72, 65,  2, 75, 66,  2, 62, 75, 80, 68, 15,
        2, 48, 75, 73, 65, 80, 69, 73, 65, 79,  2, 83, 68, 65, 74,  2, 61,
        2, 64, 69, 79, 65, 61, 79, 65,  2, 69, 79,  2, 69, 74,  2, 61, 72,
       72,  2, 80, 68, 65,  2, 73, 61, 67, 61, 86, 69, 74, 65, 79])>, <tf.Tensor: shape=(100,), dtype=int64, numpy=
array([69, 79, 79,  2, 37, 75, 75, 82, 65, 78, 27,  2, 43, 75, 13,  2, 61,
       63, 80, 81, 61, 72, 72, 85, 13,  2, 69, 80,  2, 83, 61, 79,  2, 61,
        2, 72, 69, 80, 80, 72, 65,  2, 75, 66,  2, 62, 75, 80, 68, 15,  2,
       48, 75, 73, 65, 80, 69, 73, 65, 79,  2, 83, 68, 65, 74,  2, 61,  2,
       64, 69, 79, 65, 61, 79, 65,  2, 69, 79,  2, 69, 74,  2, 61, 72, 72,
        2, 80, 68, 65,  2, 73, 61, 67, 61, 86, 69, 74, 65, 79,  2])>)


Juntemos los caracteres para que sean mas facil de visualizar.

In [20]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idxchar[input_example.numpy()])))
  print ('Target data:', repr(''.join(idxchar[target_example.numpy()])))

Input data:  'Miss Hoover: No, actually, it was a little of both. Sometimes when a disease is in all the magazines'
Target data: 'iss Hoover: No, actually, it was a little of both. Sometimes when a disease is in all the magazines '


Veamos de forma muy corta y más detallada que lo que buscamos con una sola secuencia es:

In [21]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idxchar[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idxchar[target_idx])))
print("...")

Step    0
  input: 42 ('M')
  expected output: 69 ('i')
Step    1
  input: 69 ('i')
  expected output: 79 ('s')
Step    2
  input: 79 ('s')
  expected output: 79 ('s')
Step    3
  input: 79 ('s')
  expected output: 2 (' ')
Step    4
  input: 2 (' ')
  expected output: 37 ('H')
...


### Lotes de entrenamiento

Vamos a mezclar los datos y los vamos a guardar por lotes de tamaño 64, porque así los datos no tienen dependencia de orden (silimar a la autocorrelación en la regresión) y además hacemos una actualización para cada mini lote de datos. Además las mezclan se harán en un búfer para cada bache.

In [22]:
BATCH_SIZE = 64 #tamaño del lote
BUFFER_SIZE = 500000 #bufer para mezclar el conjunto de datos

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## 4. Construcción del Modelo

Para este modelo vamos a utilizar tres capas,
   - Embegind: Es un espacio de dimensiones bajas en el que se puede traducir vectores de altas dimensiones. Tambien se considera un mapeo de objetos discretos. El embeding es obtenido internamente a partir concurrencias, proyecciones y otros.
   - GRU: (Gated recurrent unit) es un tipo de RNN compuesta por unidades GRU.
   - Dense: Todas las neuronas se conectan con todas las neuronas.

In [0]:
#Longitud del vocabulario en caracteres, tamaño de entrada(one hot)
vocab_size = len(vocab)

#Dimensión de incrustación(embedding), cada caracter sera representado por un vector de 256.
embedding_dim = 256

#numero de unidades de RNN
rnn_units = 1024

Entonces, el dataset de entrenamiento tiene lotes de tamaño 64 * 100. Notemos que en la construccion del modelo, en la capa GRU (stateful=TRUE) mantiene la memoria de la secuencia inicial y de los caracteres que van siendo generados, pero solamente se pasa en el primer paso la secuencia de entrada completa. Luego solo se pasa el caracter predicho.

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True, #regresa toda las secuencias
                        stateful=True, #cuando se pasa de un batch a otro, no se inicializan las capas intermedias
                        recurrent_initializer='glorot_uniform',
                        dropout=0.5), #forma de inicialización del estado recurrente
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True, #regresa toda las secuencias
                        stateful=True, #cuando se pasa de un batch a otro, no se inicializan las capas intermedias
                        recurrent_initializer='glorot_uniform',
                        dropout=0.5), #forma de inicialización del estado recurrente
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [0]:
#modelo
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

## 5. Configuración de checkpoints

Creamos un directorio *training_checkpoints* y utilizamos *tf.keras.callbacks.ModelCheckpoint* para guardar los puntos de control(los peses)durante el entrenamiento.

In [0]:
#directorio donde los checkpoints(puntos de control) serán guardados
checkpoint_dir = './training_checkpoints'
#nombre de los puntos de control(checkpoint files)
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix, #direccion o ruta
    save_weights_only=True) #se guardan solo los pesos

## 6. Entrenamiento

El lote de datos pasa por el modelo y devuelve un tensor de tamaño (64, 100, 65). Como el vocabulario tien 65 caracteres y cada sequencia tiene tamaño 100, el modelo predice el siguiente caracter para cada caracter en la entrada, luego el modelo asigna un valor numérico a cada elemento en el vocabulario. Cómo se elige? Se puede elegir el valor numerico máximo o se puede elegir atoriamente, que resulta ser mejor que tomar el maximo.

Para elegir aleatoriamente o tomar la muestra, la distribución es calculada a partir de las predicciones(logits) que son generadas para cada caracter del vocabulario. Para cada lote de 64 secuencias la función de pérdida se obtiene con el promedio de 64∗100=6400 entropías cruzadas dispersas.

Para compilar el modelo utilizamos la función de pérdida de entropía cruzada y el optimizardor Adam.

In [0]:
model.compile(optimizer='adam',loss=loss)

In [67]:
#en history queda registrado la funcion de perdida y la precision para el modelo, tanto en entrenamiento como en test
EPOCHS=10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
1533/1533 [==============================] - 247s 161ms/step - loss: 1.6672
Epoch 2/10
1533/1533 [==============================] - 248s 162ms/step - loss: 1.2979
Epoch 3/10
1533/1533 [==============================] - 248s 162ms/step - loss: 1.2499
Epoch 4/10
1533/1533 [==============================] - 248s 162ms/step - loss: 1.2279
Epoch 5/10
1533/1533 [==============================] - 248s 162ms/step - loss: 1.2160
Epoch 6/10
1533/1533 [==============================] - 247s 161ms/step - loss: 1.2083
Epoch 7/10
1533/1533 [==============================] - 247s 161ms/step - loss: 1.2037
Epoch 8/10
1533/1533 [==============================] - 247s 161ms/step - loss: 1.2011
Epoch 9/10
1533/1533 [==============================] - 247s 161ms/step - loss: 1.2001
Epoch 10/10
1533/1533 [==============================] - 247s 161ms/step - loss: 1.2002


## 7. Generación de texto

### Restauración del último  checkpoint

Debido a la forma en que se pasa el estado RNN de un paso a otro, el modelo solo acepta un tamaño de lote fijo una vez construido, asi que tomaremos el pre-entranamiento usando los pesos almacenados y un tamaño de lote de 1. Si se quisiera ejecutar el modelo con un tamaño de lote diferente, se necesitaría reconstruir el modelo y restaurar los pesos desde el punto de control.

In [0]:
#tomamos un solo batch
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

#utilizamos los checkpoint del pre-entrenamiento
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

#reconstruimos el modelo
model.build(tf.TensorShape([1, None]))

In [69]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (1, None, 256)            35072     
_________________________________________________________________
gru_12 (GRU)                 (1, None, 1024)           3938304   
_________________________________________________________________
gru_13 (GRU)                 (1, None, 1024)           6297600   
_________________________________________________________________
dense_9 (Dense)              (1, None, 137)            140425    
Total params: 10,411,401
Trainable params: 10,411,401
Non-trainable params: 0
_________________________________________________________________


### Predicción

¿Cómo se genera el texto?:

* Se introduce una cadena de inicio, se inicializa el estado de la  RNN y se configura el número de caracteres a generar.

* Se obtiene la distribución de predicción del siguiente carácter utilizando la cadena de inicio y el estado de la  RNN y se calcula el índice del carácter predicho. 

* Se usa este caracter predicho como la próxima entrada al modelo.

* El estado RNN devuelto retroalimenta al modelo para que ahora tenga más contexto, en lugar de una sola palabra. Después de predecir la siguiente palabra, los estados RNN modificados se retroalimentan nuevamente en el modelo, que es cómo aprende a medida que obtiene más contexto de las palabras predichas previamente.

Vamos a hacer la función para generar texto basada en los pesos obtenidos en el modelo pre-entrenado, aquí utilizamos un nuevo concepto *la temperatura* que indica que tan sorprendente o aleatorio puede ser un resultado, toma valores entre 0 y 1, donde un valor cercano a 0 indica que es mas predecible y un valor cercano a 1 indica que el resultado es más aleatorio.

In [0]:
#funcion para generar texto usando el modelo pre-entrenado
def generate_text(model, start_string, num_generate):

  #convertimos el texto inicial en numeros (vectorizacion)
  input_eval = [charxid[s] for s in start_string] #vector columna
  input_eval = tf.expand_dims(input_eval, 0) #vector fila, agrega una diension

  #guardamos el texto predicho
  text_generated = []

  temperature = 1.0

  #tamaño del bache== 1
  model.reset_states() #reinia o borra el estado recurrente de la red. Dejando valores aleatorios o ceros.
  for i in range(num_generate):
      predictions = model(input_eval)
      #removemos la dimensión del batch, quita una dimensión
      predictions = tf.squeeze(predictions, 0)

      #usamos la distribución categorica para predecir la palabra que retorna el modelo
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      #utilizamos la palabra predicha y el estado oculto anterior como entrada
      input_eval = tf.expand_dims([predicted_id], 0) #devuelve un tensor con una dimensión adicional en el eje de índice

      text_generated.append(idxchar[predicted_id])

  return (start_string + ''.join(text_generated))

Notemos que el texto generado es capaz de utilizar mayusculas, minusculas y signos de puntuación para construir frases con el vocabulario de los Simpons.

In [71]:
print(generate_text(model, start_string=u"Homer Simpson: ", num_generate=500))

Homer Simpson: You're stecking out, but I think they're section of you with some time.
Bart Simpson: (ANNOYED) Wow! (GIGGLES)
Lisa Simpson: Fine. But stop supporting a hit to us, that will come triple. This is touched to sested to bright driving and get him? And money to meet straight in!
David's tear to see until you dedrey from the electric growing.
Comic Book Guy: Marge, we're going to hear... Hello your store? I've never seen Megical Club cheese, cows... consider... make pupping season...
Young Well..


## Avanzado: Entrenamiento personalizado

Para mejorar el entrenamiento anterior y obtener más control usaremos `tf.GradientTape` bajo los siguientes pasos:

* Utilizamos el mismo modelo y el optimizador adam.

* Inicializamos el estado RNN llamando al método `tf.keras.Model.reset_states`.

* Repetimos el conjunto de datos (lote por lote) y calculamos las predicciones asociadas con cada una.

* Abrimos un `tf.GradientTape`, y calculamos las predicciones y pérdidas en ese contexto.

* Calculamos los gradientes de la pérdida con respecto a las variables del modelo utilizando el método `tf.GradientTape.grads`.

* Finalmente, damos un paso hacia abajo utilizando el método `tf.train.Optimizer.apply_gradients` del optimizador.

In [0]:
#modelo
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [0]:
#optimizador
optimizer = tf.keras.optimizers.Adam()

In [0]:
@tf.function
def train_step(inp, target): 
  with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(
            target, predictions, from_logits=True))
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

Con lo anterior hacemos el pre-entrenamiento.

In [75]:
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  #inicializamos el estado oculto al comienzo de cada época
  hidden = model.reset_states()

  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target) #usamos la funcion train_step que acabamos de creamr

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))

  #guardamos los 'checkpoint' del modelo cada 5 epocas
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

#imprimimos la perida y los pesos guardados
model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.919189929962158
Epoch 1 Batch 100 Loss 2.3017690181732178
Epoch 1 Batch 200 Loss 1.8995417356491089
Epoch 1 Batch 300 Loss 1.7622944116592407
Epoch 1 Batch 400 Loss 1.542039155960083
Epoch 1 Batch 500 Loss 1.5134451389312744
Epoch 1 Batch 600 Loss 1.4137160778045654
Epoch 1 Batch 700 Loss 1.3271125555038452
Epoch 1 Batch 800 Loss 1.4190945625305176
Epoch 1 Batch 900 Loss 1.3396272659301758
Epoch 1 Batch 1000 Loss 1.316850185394287
Epoch 1 Batch 1100 Loss 1.3055423498153687
Epoch 1 Batch 1200 Loss 1.301363229751587
Epoch 1 Batch 1300 Loss 1.191483974456787
Epoch 1 Batch 1400 Loss 1.2816455364227295
Epoch 1 Batch 1500 Loss 1.224030613899231
Epoch 1 Loss 1.2251
Time taken for 1 epoch 253.65200638771057 sec

Epoch 2 Batch 0 Loss 1.2686915397644043
Epoch 2 Batch 100 Loss 1.2236398458480835
Epoch 2 Batch 200 Loss 1.2016856670379639
Epoch 2 Batch 300 Loss 1.201305866241455
Epoch 2 Batch 400 Loss 1.1975877285003662
Epoch 2 Batch 500 Loss 1.1723922491073608
Epoch 2 Batch 

Ahora, vamos a utilizar el modelo pre-entrenado, pero vamos a utilizar un solo bache.

In [0]:
#tomamos un solo batch
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

#utilizamos los checkpoint del pre-entrenamiento
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

#reconstruimos el modelo
model.build(tf.TensorShape([1, None]))

Y utilizamos nuevamente la funcion generadora de texto.

In [0]:
#funcion para generar texto usando el modelo pre-entrenado
def generate_text(model, start_string, num_generate):

  #convertimos el texto inicial en numeros (vectorizacion)
  input_eval = [charxid[s] for s in start_string] #vector columna
  input_eval = tf.expand_dims(input_eval, 0) #vector fila, agrega una diension

  #guardamos el texto predicho
  text_generated = []

  temperature = 1.0

  #tamaño del bache== 1
  model.reset_states() #reinia o borra el estado recurrente de la red. Dejando valores aleatorios o ceros.
  for i in range(num_generate):
      predictions = model(input_eval)
      #removemos la dimensión del batch, quita una dimensión
      predictions = tf.squeeze(predictions, 0)

      #usamos la distribución categorica para predecir la palabra que retorna el modelo
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      #utilizamos la palabra predicha y el estado oculto anterior como entrada
      input_eval = tf.expand_dims([predicted_id], 0) #devuelve un tensor con una dimensión adicional en el eje de índice

      text_generated.append(idxchar[predicted_id])

  return (start_string + ''.join(text_generated))

Podemos ver que este código nos da resultados mas coherentes.

In [78]:
print(generate_text(model, u"Hello, how are you? ", 100))

Hello, how are you? Stead that money?
Maude Flafforms, Krusty's Knny, your boy.
Homer Simpson: Eh. At least there's an


### Modelos
embeding= 256
gru= 1024
dense= 138
dropout= 0.5

|embeding|gru|dense|dropout|loss|
|::|::|::|::|::|
|1|1|1||1.1060|
|1|2|1|2|1.0685|

In [0]:
## 12. Referencias

- Aprendizaje profundo, [Repositorio Ciencia-de-datos](https://github.com/AprendizajeProfundo/Ciencia-de-Datos)
- Tensorflow, [Generación de texto con una red neuronal recurrente (RNN)](https://www.tensorflow.org/tutorials/text/text_generation)
- Google, [Machine learning crash course](https://developers.google.com/machine-learning/crash-course/embeddings/video-lecture)